In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

section_dict = {
    '최신뉴스': 100,
    '정치': 200,
    '경제': 300,
    '사회': 400,
    '세계': 500,
    'IT/과학': 600
}

def print_news(section_name):
    sid = section_dict[section_name]

    if section_name == '최신뉴스':
        url = f'https://news.nate.com/recent?mid=n0{sid}'
    else:
        url = f'https://news.nate.com/section?mid=n0{sid}'

    print(f'\n===== {section_name} 뉴스 =====\nURL: {url}\n')

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    res = requests.get(url, headers=headers)
    
    if not res.ok:
        print(f"접속 실패: 상태 코드 {res.status_code}")
        return

    soup = BeautifulSoup(res.text, 'html.parser')

    # 뉴스 항목들 찾기 (최신뉴스와 섹션뉴스 동일 구조 사용)
    news_items = soup.select("div.post")

    for idx, item in enumerate(news_items[:10], 1): 
        a_tag = item.select_one("a[href*='//news.nate.com/view'].sa_text_title")
        if not a_tag:
            continue

        title = a_tag.get_text(strip=True)
        link = a_tag['href']

        print(f"{idx}. {title}")
        print(f"링크: {link}")

# 테스트 예시
for section in section_dict:
    print_news(section)



===== 최신뉴스 뉴스 =====
URL: https://news.nate.com/recent?mid=n0100


===== 정치 뉴스 =====
URL: https://news.nate.com/section?mid=n0200


===== 경제 뉴스 =====
URL: https://news.nate.com/section?mid=n0300


===== 사회 뉴스 =====
URL: https://news.nate.com/section?mid=n0400


===== 세계 뉴스 =====
URL: https://news.nate.com/section?mid=n0500


===== IT/과학 뉴스 =====
URL: https://news.nate.com/section?mid=n0600



In [31]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

# Nate 뉴스 섹션별 정확한 mid 값을 포함한 딕셔너리
section_dict = {
    '최신뉴스': '100',
    '정치': '101',
    '경제': '102',
    '사회': '103',
    '세계': '104',
    'IT/과학': '105'
}

def print_news(section_name):
    """
    주어진 Nate 뉴스 섹션에서 기사 이미지, 제목, 링크를 스크래핑하고 출력합니다.

    Args:
        section_name (str): 뉴스 섹션 이름 (예: '최신뉴스', '정치').
    """
    if section_name not in section_dict:
        print(f"오류: '{section_name}'은(는) 유효한 섹션이 아닙니다.")
        return

    mid_value = section_dict[section_name]

    # 섹션 이름에 따라 기본 URL 결정
    if section_name == '최신뉴스':
        base_url = 'https://news.nate.com/recent'
        url = f'{base_url}?mid=n0{mid_value}'
    else:
        base_url = 'https://news.nate.com/section'
        url = f'{base_url}?mid=n0{mid_value}'

    print(f'\n--- {section_name} 뉴스 ---')
    print(f'스크래핑 URL: {url}')

    # 요청 헤더 설정
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    try:
        res = requests.get(url, headers=req_header)
        res.raise_for_status() # 4xx 또는 5xx 응답에 대해 HTTPError 발생
    except requests.exceptions.RequestException as e:
        print(f"요청 실패: {e}")
        return

    print(f"상태 코드: {res.status_code}, OK: {res.ok}")

    html = res.text
    soup = BeautifulSoup(html, "html.parser")

    # 변경된 뉴스 아이템 선택자: 스크린샷을 기반으로
    # 'div.mduSubjectList' 내부의 뉴스 기사 링크(href에 '//news.nate.com/view' 포함)를 직접 선택
    news_items = soup.select('div.mduSubjectList a[href*="//news.nate.com/view"]')

    if not news_items:
        # 혹시 'postSubjectContent' 아래의 다른 리스트 형태라면
        news_items = soup.select('div.postSubjectContent a[href*="//news.nate.com/view"]')

    if not news_items:
        print("뉴스 아이템을 찾을 수 없습니다. 선택자 또는 웹사이트 구조를 다시 확인하세요.")
 
        # print("HTML Head:", soup.find('head'))
        # print("HTML Body (첫 1000자):", html[:1000])
        return

    print(f"총 {len(news_items)}개의 뉴스 아이템 발견.")

    for idx, a_tag in enumerate(news_items, 1):
        title = a_tag.text.strip() if a_tag.text else "제목 없음"
        link = a_tag['href'] if a_tag.has_attr('href') else "링크 없음"

        # 이미지 엘리먼트 찾기: a_tag 내부에서 img 태그를 찾음
        img_tag = a_tag.select_one("img")

        image_url = "이미지 없음"
        if img_tag and img_tag.has_attr('src'):
            src = img_tag['src']
            # //로 시작하는 src 경로를 올바른 절대 URL로 변환
            image_url = urljoin('https:', src)

        print(f'\n{idx}.')
        print(f'   제목: {title}')
        print(f'   링크: {link}')
        print(f'   이미지 URL: {image_url}')

        # 이미지가 있을 경우 표시
        if image_url != "이미지 없음":
            try:
                display(Image(url=image_url, width=150)) # 이미지 너비 조절 가능
            except Exception as e:
                print(f"   이미지 표시 오류 ({image_url}): {e}")
        else:
            print("   (표시할 이미지가 없습니다)")

# 모든 섹션에 대해 함수 실행
sections_to_scrape = ['최신뉴스', '정치', '경제', '사회', '세계', 'IT/과학']
for section in sections_to_scrape:
    print_news(section)


--- 최신뉴스 뉴스 ---
스크래핑 URL: https://news.nate.com/recent?mid=n0100
상태 코드: 200, OK: True
총 20개의 뉴스 아이템 발견.

1.
   제목: 감보아 7이닝 무실점 '호투'→김태형 감독 "완벽한 투구 펼쳤어" 칭찬 [SS고척in]
																	롯데 김태형 감독이 승리한 뒤 모자를 벗어 팬들에게 인사를 하고 있다. 고척 | 박진업 기자 upandup@sportsseoul.com [스포츠서울 | 고척=박연준 기자] 롯데가 선발 알렉 ...
   링크: //news.nate.com/view/20250724n41294?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/sp/2025/07/24/news-p.v1.20250724.67adc8707949458bb5608b0ccbb23ad1_P1.jpg



2.
   제목: 폭우 쏟아지는 등교길, 여고생에 우산 건넨 시민…현관문 앞서 울컥, 왜?
																	A씨가 여고생들에게 빌려준 우산(왼쪽)과 답례로 받은 복숭아와 감자. /보배드림폭우가 쏟아지던 등교 시간, 고장 난 우산으로 당황하던 여고생들에게 우산을 빌려준 40대 ...
   링크: //news.nate.com/view/20250724n41281?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ch/2025/07/24/ch_1753360264936_39787_0.jpg



3.
   제목: 이봉주, 처조카 입양해 아들처럼 키웠다…아내 "쉬운 일 아닌데 먼저 제안" 감동
																	[조이뉴스24 이지영 기자]마라토너 이봉주가 처조카를 입양해 키운 사연을 전했다.23일 방송된 tvN '유 퀴즈 온 더 블럭'에는 이봉주와 그의 아내 김미순 씨가 출연했다.이...
   링크: //news.nate.com/view/20250724n41292?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/jn/2025/07/24/451929ddeab3b4.jpg



4.
   제목: [ST포토] 노시환, 하나만 걸려라
																	[스포츠투데이 권광일 기자] 2025 신한은행 SOL Bank KBO리그 한화 이글스와 두산 베어스의 경기가 24일 서울 잠실야구장에서 열렸다. 한화 노시환이 9회초 1사2루에서 타...
   링크: //news.nate.com/view/20250724n41291?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/st/2025/07/24/24_1016930_36389.jpg



5.
   제목: 황선우와 김우민이 싱가포르에서 그릴 '어게인 도하'…"개인전과 단체전 모두 기대해"
																	한국수영대표팀이 싱가포르에서 진행 중인 2025세계수영선수권대회에서 가장 기대하는 종목은 경영이다. 황선우(22)와 김우민(24·이상 강원도청) 등 세계정상급 선수들을 ...
   링크: //news.nate.com/view/20250724n41290?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/sd/2025/07/24/132064668.1.jpg


...


17.
   제목: "내 딸 목소리도 모를까 봐?" 급하다는 울음소리, AI였다
																	미국 플로리다의 한 여성이 인공지능(AI) 기술을 활용한 보이스피싱 피해자가 됐다. 미 매체 피플에 따르면 샤론 브라이트웰은 9일(현지시간) 딸의 목소리가 AI 기술로 날...
   링크: //news.nate.com/view/20250724n41277?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mh/2025/07/24/news-p.v1.20250723.404113884c3541cc8f5bed85a57254af_P1.png



18.
   제목: 김혜경 여사, 가평 수해 현장 찾아 자원봉사…"진심 담아 위로"
																	이재명 대통령의 배우자인 김혜경 여사가 집중호우로 피해를 입은 경기도 가평군 수해복구 현장을 찾아 봉사활동을 했습니다. 대통령실에 따르면 김 여사는 오늘(24일) 비...
   링크: //news.nate.com/view/20250724n41276?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ny/2025/07/24/AKR20250724212928059_03_i.jpg



19.
   제목: 전교조, 넷플릭스 시리즈 '참교육' 제작 중단 촉구…"교사 모욕"
																	교원과 시민단체가 공무원이 문제 학생과 교사 등을 폭행하는 내용을 담은 드라마 ‘참교육’의 제작을 중단하라고 촉구했다. 전국교직원노동조합(전교조)과 평등교육실현...
   링크: //news.nate.com/view/20250724n41275?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/mh/2025/07/24/news-p.v1.20250723.00fdf0bcfd5e46c99dfbf1fae5d1d429_P1.jpg



20.
   제목: [MD포토] 홍민기 '승리 미소'
																	[마이데일리 = 유진형 기자] 24일 오후 서울 고척스카이돔에서 열린 '2025 신한 SOL뱅크 KBO 리그' 롯데 자이언츠와 키움 히어로즈의 경기.롯데 홍민기가 4-0으로 승리한 ...
   링크: //news.nate.com/view/20250724n41274?mid=n0100
   이미지 URL: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/my/2025/07/24/2025072421295367922_l.jpg



--- 정치 뉴스 ---
스크래핑 URL: https://news.nate.com/section?mid=n0101
상태 코드: 200, OK: True
뉴스 아이템을 찾을 수 없습니다. 선택자 또는 웹사이트 구조를 다시 확인하세요.

--- 경제 뉴스 ---
스크래핑 URL: https://news.nate.com/section?mid=n0102
상태 코드: 200, OK: True
뉴스 아이템을 찾을 수 없습니다. 선택자 또는 웹사이트 구조를 다시 확인하세요.

--- 사회 뉴스 ---
스크래핑 URL: https://news.nate.com/section?mid=n0103
상태 코드: 200, OK: True
뉴스 아이템을 찾을 수 없습니다. 선택자 또는 웹사이트 구조를 다시 확인하세요.

--- 세계 뉴스 ---
스크래핑 URL: https://news.nate.com/section?mid=n0104
상태 코드: 200, OK: True
뉴스 아이템을 찾을 수 없습니다. 선택자 또는 웹사이트 구조를 다시 확인하세요.

--- IT/과학 뉴스 ---
스크래핑 URL: https://news.nate.com/section?mid=n0105
상태 코드: 200, OK: True
뉴스 아이템을 찾을 수 없습니다. 선택자 또는 웹사이트 구조를 다시 확인하세요.
